In [1]:
import numpy as np
import tensorflow as tf
import random as rn
import os
from my_utils.preprocess import DataPreprocess
from my_utils.nfold_test import NFoldModel
from my_utils.dataloader import Workout_dataset
from my_utils.model import CNN_RNN_Resnet

In [2]:
# seed 고정
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(seed)
np.random.seed(seed)
rn.seed(seed)

In [3]:
features_path = './data/train_features.csv'
labels_path = './data/train_labels.csv'

preprocess = DataPreprocess(
    folds=10,seed=seed,acc_norm=10,gy_norm=2000,
    features_path=features_path,labels_path=labels_path)

In [4]:
preprocess.labels_to_csv()
preprocess.features_to_npy()

In [5]:
nfold_model = NFoldModel()
nfold_model.Model_class = CNN_RNN_Resnet
nfold_model.Workout_dataset = Workout_dataset
nfold_model.model_parameter = dict(leakyrelu_alpha = 0.1, 
    input_kernels = 20, input_kernel_width = 3, input_regularize_coeff=0.001,
    res_kernels = 60, res_kernel_width = 3, res_regularize_coeff=0.1, res_num = 5
    )

In [6]:
fold_list = [0,5,9]
nfold_model.nfold_train(fold_list)

.hdf5
Epoch 14/2000
35/35 [==============================] - 15s 428ms/step - loss: 3.2561 - accuracy: 0.6839 - val_loss: 4.9434 - val_accuracy: 0.4766

Epoch 00014: val_loss did not improve from 4.71584
Epoch 15/2000
35/35 [==============================] - 15s 422ms/step - loss: 3.0775 - accuracy: 0.7156 - val_loss: 4.9865 - val_accuracy: 0.2227

Epoch 00015: val_loss did not improve from 4.71584
Epoch 16/2000
35/35 [==============================] - 15s 421ms/step - loss: 2.9941 - accuracy: 0.6821 - val_loss: 6.2389 - val_accuracy: 0.0625

Epoch 00016: val_loss did not improve from 4.71584
Epoch 17/2000
35/35 [==============================] - 15s 421ms/step - loss: 2.8487 - accuracy: 0.6902 - val_loss: 14.4065 - val_accuracy: 0.0703

Epoch 00017: val_loss did not improve from 4.71584

Epoch 00017: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 18/2000
35/35 [==============================] - 15s 428ms/step - loss: 2.5732 - accuracy: 0.7241 - val_loss: 3.34

In [7]:
test_batch_size = 625
for fold in fold_list:
    nfold_model.evaluate_test(test_batch_size,fold)

1/1 [==============================] - 19s 19s/step - loss: 0.9376 - accuracy: 0.8768


In [8]:
predict = nfold_model.nfold_predict(test_batch_size,fold_list)

In [16]:
predict_labels = np.argmax(predict,axis=1)
test_labels = preprocess.y_test

In [17]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
accuracy_score(test_labels,predict_labels),recall_score(test_labels,predict_labels,average='macro'), precision_score(test_labels,predict_labels,average='macro'),f1_score(test_labels,predict_labels,average='macro')

(0.872, 0.7386452812358766, 0.8041132560475291, 0.7479661953082858)